In [66]:
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

serial exception on close write failed: [Errno 6] Device not configured
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [51]:
import machine
print(gc.mem_free())
machine.reset()

65440
ets Jul 29 2019 12:21:46

rst:0xc (SW_CPU_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4252
load:0x40078000,len:11920
load:0x40080400,len:3344
entry 0x4008060c
WebREPL daemon started on ws://0.0.0.0:8266
Started webrepl in normal mode
Traceback (most recent call last):
  File "main.py", line 31
IndentationError: unindent doesn't match any outer indent level
MicroPython 97a7cc2-dirty on 2021-11-26; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]normal repl mode not detected b''
not command mode

>>> 
>>> 
raw REPL; CTRL-B to exit
> 

In [52]:
from ulab import numpy as np
import gc

from lib.computation import solve_eig_qr, standardise, solve_gen_eig_prob
    
class CCA():
    
    def __init__(self, stim_freqs, fs, Nh=2):
        self.Nh = Nh
        self.stim_freqs = stim_freqs
        self.fs = fs
        
    def compute_corr(self, X_test):            
        result = {}
        Cxx = np.dot(X_test, X_test.transpose()) # precompute data auto correlation matrix
        for f in self.stim_freqs:
            Y = harmonic_reference(f, self.fs, np.max(X_test.shape), Nh=self.Nh, standardise_out=False)
            rho = self.cca_eig(X_test, Y, Cxx=Cxx) # canonical variable matrices. Xc = X^T.W_x
            result[f] = rho
        return result
    
    @staticmethod
    def cca_eig(X, Y, Cxx=None, eps=1e-6):
        if Cxx is None:
            Cxx = np.dot(X, X.transpose()) # auto correlation matrix
        Cyy = np.dot(Y, Y.transpose()) 
        Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
        Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

        M1 = np.dot(np.linalg.inv(Cxx+eps), Cxy) # intermediate result
        M2 = np.dot(np.linalg.inv(Cyy+eps), Cyx)

        lam, _ = solve_eig_qr(np.dot(M1, M2), 20)
        return np.sqrt(lam)
    
class UnivariateMsetCCA():
    """
    Multiset CCA algorithm for SSVEP decoding.
    
    Computes optimised reference signal set based on historical observations
    and uses ordinary CCA for final correlation computation given a new test
    signal.
    
    Note: this is a 1 channel implementation (Nc=1)
    """
    
    def __init__(self):
        self.Ns, self.Nt = None, None
        
    def fit(self, X, compress_ref=True):
        """
        Expects a training matrix X of shape Nt x Ns. If `compress_ref=True`, the `Nt` components
        in optimised reference signal Y will be averaged to form a single reference vector. This
        can be used for memory optimisation but will likely degrade performance slightly.         
        """
        if X.shape[0] > X.shape[1]:
            print("Warning: received more trials than samples. This is unusual behaviour: check orientation of X"
                 )
        R = np.dot(X, X.transpose()) # inter trial covariance matrix
        S = np.eye(len(R))*np.diag(R) # intra-trial diag covariance matrix

        lam, V = solve_gen_eig_prob((R-S), S) # solve generalised eig problem
        w = V[:, np.argmax(lam)] # find eigenvector corresp to largest eigenvalue
        Y = np.array([x*w[i] for i, x in enumerate(X)]) # store optimised reference vector Nt x Ns
        self.Y  = Y
        if compress_ref:
            self.Y = np.mean(Y, axis=0).reshape((1, max(Y.shape))) # this will average Nt components in Y: Nc x Nt -> 1 x Nt
        
    def compute_corr(self, X_test):
        if self.Y is None:
            raise ValueError("Reference matrix Y must be computed using `fit` before computing corr")
        if len(X_test.shape) == 1:
            X_test = X_test.reshape((1, len(X_test)))
        return CCA.cca_eig(X_test, self.Y)[0]
          

def harmonic_reference(f0, fs, Ns, Nh=1, standardise_out=False):
    
    '''
    Generate reference signals for canonical correlation analysis (CCA)
    -based steady-state visual evoked potentials (SSVEPs) detection [1, 2].
    function [ y_ref ] = cca_reference(listFreq, fs,  Ns, Nh) 
    Input:
      f0        : stimulus frequency
      fs              : Sampling frequency
      Ns              : # of samples in trial
      Nh          : # of harmonics
    Output:
      y_ref           : Generated reference signals with shape (Nf, Ns, 2*Nh)
    '''  
    X = np.zeros((Nh*2, Ns))
    
    for harm_i in range(Nh):
        # Sin and Cos
        X[2*harm_i, :] = np.sin(np.arange(1,Ns+1)*(1/fs)*2*np.pi*(harm_i+1)*f0)
        gc.collect()
        X[2*harm_i+1, :] = np.cos(np.arange(1,Ns+1)*(1/fs)*2*np.pi*(harm_i+1)*f0)
        gc.collect()

    # print(micropython.mem_info(1))
    if standardise_out: # zero mean, unit std. dev
        return standardise(X)
    return X


In [53]:
data =  np.array([               0.1708755,
                10.43701,
                22.7063,
                -126.6594,
                -220.1675,
                162.305,
                68.75163,
                -237.0515,
                163.1026,
                -70.54583,
                -201.3133,
                237.1926,
                170.6061,
                -54.52446,
                -61.22984,
                92.65961,
                -80.15914,
                19.30758,
                32.94912,
                -35.16199,
                91.69647,
                -99.0492,
                -39.99503,
                62.33409,
                144.7027,
                -104.1411,
                -105.7084,
                127.5001,
                -70.36548,
                -299.6768,
                -269.9744,
                415.2166,
                48.08802,
                -272.515,
                165.5221,
                -359.8768,
                -159.1279,
                375.0938,
                226.8854,
                -19.62947,
                62.06848,
                -200.2913,
                -317.8484,
                268.766,
                -79.12334,
                -27.75001,
                190.3486,
                -88.47918,
                -201.5865,
                176.2455,
                654.6741,
                -143.7516,
                -97.50078,
                93.91964,
                -515.5127,
                -176.8522,
                279.1294,
                183.0208,
                -50.89166,
                37.90892,
                -202.9223,
                69.85486,
                377.9689,
                -169.2203,
                0.1100412,
                2.203442,
                -17.70377,
                -87.92948,
                -66.27866,
                38.17255,
                -39.46883,
                -59.05927,
                49.5677,
                -110.6703,
                -352.5143,
                -304.4713,
                112.3953,
                90.42348,
                -279.9076,
                -55.61628,
                -95.42358,
                -194.1472,
                -54.49891,
                -66.10148,
                2.431595,
                -30.59612,
                -54.42813,
                -278.6809,
                -54.72312,
                276.7576,
                -116.7799,
                -206.5133,
                -89.73518,
                -335.6983,
                -418.6193,
                -20.06284,
                45.12387,
                -98.6022,
                53.88667,
                -172.2332,
                -175.2598,
                234.7633,
                -9.379055,
                -304.0945,
                -59.59173,
                -42.60774,
                -368.5062,
                -575.4944,
                -488.015,
                -1.39946,
                763.4699,
                799.706,
                157.6303,
                750.2317,
                744.6054,
                -138.8296,
                400.1875,
                66.74978,
                -224.0326,
                275.8531,
                203.1642,
                226.879,
                107.8747,
                61.56484,
                -204.0803,
                9.715576,
                416.2397,
                99.41525,
                0.04279981,
                1.286755,
                9.638443,
                -56.88752,
                -293.9321,
                1.721092,
                648.1969,
                97.35316,
                -130.5178,
                101.6215,
                -491.0769,
                92.57454,
                286.2556,
                -239.3685,
                152.7437,
                104.4826,
                -87.26619,
                -52.6085,
                244.4329,
                184.3575,
                -3.061832,
                138.7164,
                -262.2638,
                -191.2556,
                195.0494,
                55.50439,
                -110.6812,
                90.87771,
                22.31548,
                -357.3407,
                -36.63191,
                262.5456,
                101.0789,
                10.36746,
                -40.39317,
                -149.6502,
                170.1383,
                235.0184,
                -143.5637,
                23.58678,
                106.0261,
                38.53388,
                17.3125,
                16.42669,
                65.72305,
                92.20525,
                33.54072,
                -242.35,
                -695.4861,
                -1293.525,
                -964.9776,
                -406.0865,
                23.50393,
                830.1847,
                115.5608,
                -61.31123,
                716.6125,
                0.3975983,
                38.72186,
                191.5816,
                -63.1669,
                180.2577,
                68.17661,
                207.7401,
                0.1509864,
                6.22355,
                -11.62364,
                -202.1047,
                -266.8439,
                151.7633,
                99.07786,
                -82.10938,
                259.5661,
                -93.79149,
                -83.28936,
                387.1134,
                100.9176,
                -54.25454,
                0.1175485,
                -33.50515,
                24.17977,
                174.1238,
                -39.70542,
                -59.43925,
                139.9876,
                -77.55615,
                -37.90219,
                104.3595,
                131.3247,
                45.39882,
                -67.10987,
                -65.73207,
                -30.80589,
                57.43994,
                29.00897,
                2.748097,
                9.806821,
                79.39011,
                -85.26965,
                -326.2305,
                -39.47812,
                270.7134,
                -3.491774,
                -157.8717,
                51.20653,
                -124.8461,
                -97.9975,
                55.52942,
                27.74609,
                71.91869,
                63.63812,
                -103.6375,
                -165.2847,
                433.8666,
                285.7737,
                -252.7988,
                110.0588,
                -92.93729,
                -495.433,
                -36.90336,
                358.0224,
                79.91867,
                -8.129162,
                23.03158,
                -545.5321,
                -560.0782,
                180.7078,
                389.5178
            ])
   

data1 = np.array([
               0.287932,
                19.26353,
                90.98708,
                79.73335,
                -41.28042,
                54.89269,
                95.95376,
                -127.2207,
                -52.77982,
                252.2376,
                140.4439,
                -154.0108,
                20.75153,
                -6.476227,
                -251.0048,
                -78.10915,
                148.6734,
                130.7541,
                3.924419,
                -24.18994,
                -74.83491,
                147.8979,
                122.8119,
                -149.4151,
                35.84116,
                55.25503,
                -107.3436,
                -29.65011,
                206.2439,
                69.13632,
                -109.6547,
                -20.31715,
                -126.4602,
                -33.88626,
                106.6793,
                19.48067,
                60.2982,
                144.2356,
                -21.43948,
                -291.7273,
                -107.5152,
                204.9281,
                17.95557,
                -149.2565,
                57.21002,
                -13.61454,
                -98.6378,
                122.8722,
                169.5126,
                39.56566,
                -23.46212,
                -102.3459,
                -287.4186,
                -20.97639,
                78.06087,
                -105.2728,
                60.98433,
                112.6868,
                -121.9146,
                -171.3056,
                338.0892,
                210.0135,
                -234.3968,
                0.8642998,
                0.05566993,
                0.232897,
                13.90674,
                92.14377,
                127.4871,
                -69.01407,
                -161.6378,
                7.436059,
                1.21373,
                -146.0972,
                -57.51271,
                193.7071,
                45.52183,
                -174.9929,
                -121.8434,
                -242.6754,
                -146.6342,
                80.44197,
                -35.89631,
                -41.6489,
                53.63575,
                -202.7198,
                -228.3259,
                130.3104,
                115.2177,
                -139.8264,
                -22.20526,
                39.34638,
                -239.8977,
                -117.1896,
                133.3182,
                121.3433,
                15.28212,
                -69.9093,
                -290.3603,
                -194.2134,
                208.8342,
                -41.40113,
                -114.4835,
                174.5512,
                -96.03762,
                -296.3105,
                157.201,
                227.6699,
                -160.7647,
                -94.49626,
                -18.25835,
                -102.5243,
                -18.89247,
                17.25617,
                40.83119,
                62.62219,
                -2.433503,
                -310.131,
                -652.9781,
                -372.0571,
                -52.34061,
                481.1298,
                1016.111,
                126.8778,
                -90.4602,
                586.6583,
                241.1669,
                267.523,
                0.1118117,
                11.94406,
                47.76896,
                -11.32149,
                -53.29326,
                178.3053,
                98.76609,
                -125.49,
                42.91799,
                5.773598,
                32.45723,
                130.7167,
                95.16929,
                108.4496,
                65.53422,
                -238.4152,
                -948.3442,
                -1167.393,
                -790.7556,
                -506.8366,
                -16.71439,
                664.3608,
                333.215,
                211.7301,
                464.5657,
                -80.88192,
                270.6945,
                180.8134,
                -276.1187,
                126.664,
                341.9281,
                214.201,
                -111.6598,
                90.49384,
                -192.5555,
                -252.732,
                134.1853,
                59.91457,
                111.3317,
                80.3683,
                38.90313,
                -219.9625,
                86.25055,
                234.2001,
                -87.69953,
                122.9775,
                191.771,
                81.83473,
                127.7662,
                186.2214,
                21.93783,
                11.44332,
                -7.243879,
                12.95197,
                48.52893,
                94.42107,
                177.7193,
                23.79834,
                106.4093,
                -236.2516,
                -488.8141,
                39.45769,
                180.4726,
                -177.8012,
                0.1708755,
                10.43701,
                22.7063,
                -126.6594,
                -220.1675,
                162.305,
                68.75163,
                -237.0515,
                163.1026,
                -70.54583,
                -201.3133,
                237.1926,
                170.6061,
                -54.52446,
                -61.22984,
                92.65961,
                -80.15914,
                19.30758,
                32.94912,
                -35.16199,
                91.69647,
                -99.0492,
                -39.99503,
                62.33409,
                144.7027,
                -104.1411,
                -105.7084,
                127.5001,
                -70.36548,
                -299.6768,
                -269.9744,
                415.2166,
                48.08802,
                -272.515,
                165.5221,
                -359.8768,
                -159.1279,
                375.0938,
                226.8854,
                -19.62947,
                62.06848,
                -200.2913,
                -317.8484,
                268.766,
                -79.12334,
                -27.75001,
                190.3486,
                -88.47918,
                -201.5865,
                176.2455,
                654.6741,
                -143.7516,
                -97.50078,
                93.91964,
                -515.5127,
                -176.8522,
                279.1294,
                183.0208,
                -50.89166,
                37.90892,
                -202.9223,
                69.85486,
                377.9689,
                -169.2203
            ])

test = np.array([
                0.1044539,
                9.449228,
                47.52682,
                27.9067,
                -90.04919,
                -33.83585,
                45.41429,
                -124.2994,
                -21.14599,
                152.9298,
                -4.664166,
                -69.40703,
                -112.9218,
                -32.11887,
                98.67253,
                14.44653,
                -10.64262,
                67.42153,
                20.44796,
                -251.0738,
                -115.4137,
                167.7059,
                29.19091,
                -86.59068,
                -41.72652,
                -5.722195,
                -9.977262,
                27.45411,
                55.9483,
                88.11945,
                57.82787,
                -50.7624,
                -177.2297,
                -7.996485,
                154.218,
                -108.045,
                -66.79147,
                111.8733,
                -65.52345,
                -124.2731,
                128.3,
                198.1181,
                -17.92068,
                -28.70894,
                -80.45231,
                -210.4116,
                59.00922,
                89.94083,
                -98.84078,
                78.10653,
                97.1464,
                -155.1387,
                -51.58758,
                271.2256,
                127.8247,
                -78.84561,
                34.37788,
                -56.28049,
                -55.41024,
                97.7654,
                33.08029,
                40.66964,
                67.21152,
                -40.42142,
                -0.4562313,
                -50.75227,
                -396.0128,
                -857.835,
                -548.2339,
                -185.1201,
                -507.9252,
                -323.2451,
                -30.87212,
                393.9044,
                644.7596,
                -34.58191,
                181.1036,
                337.1735,
                16.81338,
                54.87392,
                30.92483,
                364.3346,
                16.52837,
                -116.6685,
                171.6594,
                -103.2801,
                -138.3748,
                -7.798939,
                212.5956,
                68.35383,
                102.3601,
                61.04872,
                -66.92981,
                221.3286,
                -35.76456,
                -64.30923,
                71.35406,
                83.88788,
                103.2732,
                12.28451,
                110.3612,
                90.77971,
                -23.9362,
                -150.7874,
                -54.31845,
                0.5414848,
                -205.8403,
                -183.4131,
                -97.10696,
                -243.8009,
                -516.6613,
                -275.9896,
                522.3033,
                560.3996,
                67.75001,
                104.0325,
                -43.18155,
                42.61898,
                324.4141,
                46.01398,
                118.2924,
                6.973361,
                -47.81731,
                217.6779,
                -8.298174,
                -76.12126,
                51.57341,
                68.95459,
                0.04502541,
                2.677477,
                25.54125,
                94.79638,
                136.102,
                48.47864,
                -45.26698,
                -100.1822,
                -179.4717,
                -92.93952,
                35.62926,
                -50.07685,
                -1.099104,
                93.37779,
                -91.19009,
                -142.7771,
                108.4996,
                134.4473,
                -184.6347,
                -103.9956,
                -18.91737,
                -225.6531,
                -128.5465,
                58.42136,
                79.01512,
                -24.45014,
                -40.99227,
                -192.4563,
                -16.07694,
                219.4678,
                -52.56986,
                -16.1242,
                115.2998,
                -71.18688,
                -174.9377,
                156.1943,
                169.0203,
                -45.23743,
                31.48208,
                -64.07338,
                -43.68898,
                91.72104,
                70.82582,
                3.501778,
                33.44995,
                44.29779,
                -119.0529,
                -62.20986,
                204.3832,
                70.20073,
                -155.2356,
                67.94775,
                58.66019,
                -155.668,
                -20.0054,
                232.8759,
                129.6659,
                -100.9896,
                -3.203524,
                -89.54294,
                -58.72017,
                109.5342,
                71.05589,
                104.1814,
                0.07250508,
                0.4901917,
                -34.58108,
                -183.7736,
                -202.4011,
                75.14783,
                46.78237,
                -16.86756,
                167.1035,
                -84.85178,
                -42.20053,
                270.6569,
                47.88031,
                -135.5883,
                -3.058017,
                76.1953,
                -84.06169,
                -61.94026,
                125.2714,
                88.13979,
                -48.15418,
                -91.91517,
                -41.04964,
                95.2332,
                92.85108,
                -30.10443,
                78.5172,
                80.84392,
                -138.1482,
                -185.6812,
                50.27436,
                118.3052,
                -141.2272,
                -53.74664,
                41.38608,
                -98.91831,
                40.33528,
                134.3459,
                92.58174,
                32.29435,
                -11.61603,
                -172.6843,
                -200.2806,
                88.48941,
                15.78694,
                -70.34671,
                128.191,
                46.64526,
                -205.5397,
                83.70833,
                340.8249,
                -61.41984,
                -123.8647,
                47.83408,
                -159.9744,
                -221.0913,
                36.49785,
                161.1505,
                39.1115,
                0.4491711,
                -76.52993,
                1.8365,
                206.0924,
                -22.23691])

train = np.array([data,data1])
# train.append(data)
# train.append(data1)

In [54]:
print(gc.mem_free())
print(train)

33648
array([[0.1708755, 10.43701, 22.7063, ..., -560.0782, 180.7078, 389.5178],
       [0.287932, 19.26353, 90.98708, ..., 69.85486, 377.9689, -169.2203]], dtype=float32)


In [62]:
mset10hz = UnivariateMsetCCA()
mset10hz.fit(train, False)

In [63]:
print(gc.mem_free())

59584


In [64]:
print(mset10hz.compute_corr(test))

0.1033217


In [65]:
gc.collect()